In [1]:
import os
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
import PIL

from torchsummary import summary

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import configparser as ConfigParser


import utils
# Ipmport various network architectures
from networks import AGRadGalNet, VanillaLeNet, testNet, DNSteerableLeNet, DNSteerableAGRadGalNet
# Import various data classes
from datasets import FRDEEPF
from datasets import MiraBest_full, MBFRConfident, MBFRUncertain, MBHybrid
from datasets import MingoLoTSS, MLFR, MLFRTest

# Set seeds for reproduceability
torch.manual_seed(42)
np.random.seed(42)

# Get correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
# Read in config file
config_name = ["bowles2021mirabest.cfg", 
               "bowles2021LoTSS.cfg", 
               "scaife2021mirabest.cfg", 
               "scaife2021mirabestVanilla.cfg",
               "e2attentionmirabest.cfg",
               "test.cfg"
              ]
config_name = "configs/"+config_name[0]
config = ConfigParser.ConfigParser(allow_no_value=True)
config.read(config_name)

['configs/bowles2021mirabest.cfg']

In [4]:
print(np.arange(1,11,2)//2)
print(np.arange(1,11,2))


[0 1 2 3 4]
[ 0  2  4  6  8 10]


In [2]:
quiet = config.getboolean('DEFAULT', 'quiet')
early_stopping = config.getboolean('training', 'early_stopping')

# Read / Create Folder for Data to be Saved
root = config['data']['directory']
os.makedirs(root, exist_ok=True)

# -----------------------------------------------------------------------------
# Load network architecture (with random seeded weights)
print(f"Loading in {config['model']['base']}")
net = locals()[config['model']['base']](**config['model']).to(device)

if not quiet:
    if 'DN' not in config['model']['base']:
        summary(net, (1, 150, 150))
    print(device)
    if device == torch.device('cuda'):
        print(torch.cuda.get_device_name(device=device))

# -----------------------------------------------------------------------------
# Create data transformations
datamean = config.getfloat('data', 'datamean')
datastd = config.getfloat('data', 'datastd')
number_rotations = config.getint('data', 'number_rotations')
imsize = config.getint('data', 'imsize')
scaling_factor = config.getfloat('data', 'scaling')
angles = np.linspace(0, 359, config.getint('data', 'number_rotations'))
p_flip = 0.5 if config.getboolean('data','flip') else 0
augment = config.getboolean('data', 'augment')

# Create hard random (seeded) rotation:
class RotationTransform:
    """Rotate by one of the given angles."""
    def __init__(self, angles, interpolation):
        self.angles = angles
        self.interpolation = interpolation

    def __call__(self, x):
        angle = np.random.choice(a=self.angles, size=1)[0]
        return transforms.functional.rotate(x, angle, resample=self.interpolation)

# Compose dict of transformations
transformations = {
    'none': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([datamean],[datastd])
    ]),
    'rotation and flipping': transforms.Compose([
        transforms.CenterCrop(imsize),
        transforms.RandomVerticalFlip(p=p_flip),
        RotationTransform(angles, interpolation=PIL.Image.BILINEAR),
        transforms.RandomAffine(
            degrees=0, # No uncontrolled rotation
            scale=(1-scaling_factor, 1+scaling_factor), 
            resample=PIL.Image.BILINEAR),
        transforms.ToTensor(),
        transforms.Normalize([datamean],[datastd])
    ]),
    'no rotation no flipping': transforms.Compose([
        transforms.CenterCrop(imsize),
        transforms.RandomVerticalFlip(p=p_flip),
        transforms.RandomAffine(
            degrees=0, # No uncontrolled rotation
            scale=(1-scaling_factor, 1+scaling_factor), 
            resample=PIL.Image.BILINEAR),
        transforms.ToTensor(),
        transforms.Normalize([datamean],[datastd])
    ])
}

download = True
train = True
data_class = locals()[config['data']['dataset']]

if augment and p_flip==0.5:
    transform = transformations['rotation and flipping']
else:
    transform = transformations['no rotation no flipping']

train_data = data_class(root=root, download=download, train=True, transform=transform)

# -----------------------------------------------------------------------------
# Cross Validation Parameters
def fetch_grid_search_param(name, config=config):
    raw_ = config['grid_search'][name]
    raw_list = raw_.split(',')
    do = bool(raw_list.pop(0))
    out = np.asarray(raw_list, dtype=np.float64)
    return do, out

# -----------------------------------------------------------------------------
# Get data parameters
batch_size = config.getint('training', 'batch_size')
validation_size = config.getfloat('training', 'validation_set_size')
dataset_size = len(train_data)

nval = int(validation_size*dataset_size)
indices = list(range(dataset_size))
np.random.shuffle(indices)

# -----------------------------------------------------------------------------
# Split Training Data for (Cross) Validation
train_indices, val_indices = indices[nval:], indices[:nval]
train_sampler = torch.utils.data.Subset(train_data, train_indices)
valid_sampler = torch.utils.data.Subset(train_data, val_indices)

train_loader = torch.utils.data.DataLoader(train_sampler, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_sampler, batch_size=batch_size, shuffle=True)

# -----------------------------------------------------------------------------
# Extract learning values
learning_rate = config.getfloat('training', 'learning_rate')

do, lr_scaling = fetch_grid_search_param(name='learning_rate', config=config)
learning_rate *= lr_scaling

optim_name = config['training']['optimizer']

Loading in AGRadGalNet
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 6, 150, 150]              60
              ReLU-2          [-1, 6, 150, 150]               0
       BatchNorm2d-3          [-1, 6, 150, 150]              12
            Conv2d-4          [-1, 6, 150, 150]             330
              ReLU-5          [-1, 6, 150, 150]               0
       BatchNorm2d-6          [-1, 6, 150, 150]              12
            Conv2d-7          [-1, 6, 150, 150]             330
              ReLU-8          [-1, 6, 150, 150]               0
       BatchNorm2d-9          [-1, 6, 150, 150]              12
        MaxPool2d-10            [-1, 6, 75, 75]               0
           Conv2d-11           [-1, 16, 75, 75]             880
             ReLU-12           [-1, 16, 75, 75]               0
      BatchNorm2d-13           [-1, 16, 75, 75]              32
           Conv2

/raid/scratch/mbowles/EquivariantSelfAttention/venv/lib/python3.8/site-packages/torch/nn/functional.py:3451: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
/raid/scratch/mbowles/EquivariantSelfAttention/venv/lib/python3.8/site-packages/torchvision/transforms/transforms.py:1314: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  warnings.warn(


In [ ]:
# -----------------------------------------------------------------------------
# Training function test
weight_decay = config.getfloat('training', 'weight_decay')
lr = config.getfloat('final_parameters', 'learning_rate')
optimizers = {
    'SGD': optim.SGD(net.parameters(), lr=lr, momentum=0.9),
    'Adagrad': optim.Adagrad(net.parameters(), lr=lr),
    'Adadelta': optim.Adadelta(net.parameters(), lr=lr),
    'Adam': optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)
    }
optimizer  = optimizers[optim_name]
model, conf_mat, validatopn_min = utils.train(
    net,
    device,
    config,
    train_loader,
    valid_loader,
    optimizer=optimizer,
    root_out_directory_addition=f'',
    scheduler = None,
    save_validation_updates=True,
    class_splitting_index=1,
    loss_function=nn.CrossEntropyLoss(),
    output_model=True,
    early_stopping=True,
    output_best_validation=True        
)
print(f"""Confusion Matrix: {conf_mat}
Learning Rate: {lr}
""")

Folder to be saved to: models/bowles2021/mirabest
Is PATH: True


/raid/scratch/mbowles/EquivariantSelfAttention/venv/lib/python3.8/site-packages/torchvision/transforms/functional.py:935: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  warnings.warn(


Epoch:  0	Training Loss: 0.823907		Validation Loss: 0.854741
	Validation Loss Down: 	(     inf-->0.854741) ... Updating saved model.
Epoch:  1	Training Loss: 0.820954		Validation Loss: 0.852446
	Validation Loss Down: 	(0.854741-->0.852446) ... Updating saved model.
Epoch:  2	Training Loss: 0.820517		Validation Loss: 0.858517
Epoch:  3	Training Loss: 0.821320		Validation Loss: 0.856533
Epoch:  4	Training Loss: 0.820001		Validation Loss: 0.858293
Epoch:  5	Training Loss: 0.816358		Validation Loss: 0.853882
Epoch:  6	Training Loss: 0.810323		Validation Loss: 0.854228
Epoch:  7	Training Loss: 0.809393		Validation Loss: 0.848633
	Validation Loss Down: 	(0.852446-->0.848633) ... Updating saved model.
Epoch:  8	Training Loss: 0.805931		Validation Loss: 0.837892
	Validation Loss Down: 	(0.848633-->0.837892) ... Updating saved model.
Epoch:  9	Training Loss: 0.808654		Validation Loss: 0.852138
Epoch: 10	Training Loss: 0.799835		Validation Loss: 0.845425
Epoch: 11	Training Loss: 0.791415		Valida